## Visualizing Institutional Research Using Open Data

**Code for data collection and cleaning**

(Note: I am a research support librarian who's still learning Python for bibliometric analysis. Feel free to [email me](mailto:lbaster@ust.hk) if you spot any errors. Much appreciated :))

### Table of Content

**[S1: How are we progressing toward open access?](#S1)**<br>
**[S2: Which journals have we published in the most over the last 10 years?](#S1)**<br>
**[S3: How has our contribution to the Sustainable Development Goals (SDG) evolved over the years?](#S1)**<br>
**[S4: Which of our publications have had the most impact over the years?](#S1)**<br>

---

### <a id="S1">S1: How are we progressing toward open access?</a>

#### HKUST journal & conference articles, 2014-2023, by OA status

In [ ]:
import requests
import pandas as pd

# Create a function to build the API URL
def build_url(base_url, endpoint, filters, group_by=None):
    filters_str = ",".join(f"{key}:{value}" for key, value in filters.items())
    url = f"{base_url}/{endpoint}?filter={filters_str}"
    if group_by:
        url += f"&group_by={group_by}"
    return url

# Create a function to get data from the API
def get_data_from_api(url):
    response = requests.get(url)
    if response.ok:
        data = response.json()
        return data.get('group_by') or data
    return []

# Create a function to create and merge data frames
def create_and_merge_dfs(data_info):
    dfs = [
        pd.DataFrame(data)[['key', 'count']]
            .rename(columns={'key': 'publication_year', 'count': col_name})
            .fillna(0)
        for data, col_name in data_info
    ]

    merged_df = dfs[0]
    for df in dfs[1:]:
        merged_df = pd.merge(merged_df, df, on='publication_year', how='outer').fillna(0)
    return merged_df

In [ ]:
# Parameters
base_url = 'https://api.openalex.org'
endpoint = 'works'
ror_id = '00q4vv597'  # HKUST ROR
pub_year_range = '2014-2023'
pub_type = 'article'
source_type = 'journal|conference'
group_by = 'publication_year'

# Common filters
common_filters = {
    'institutions.ror': ror_id,
    'publication_year': pub_year_range,
    'type': pub_type,
    'primary_location.source.type': source_type,
    'is_paratext': 'false'
}

In [ ]:
# OA Statuses
oa_statuses = ['gold', 'hybrid', 'bronze', 'green', '!gold|!hybrid|!bronze']

# Column name mapping
column_names = {
    'gold': 'gold_oa',
    'hybrid': 'hybrid_oa',
    'bronze': 'bronze_oa',
    'green': 'green_oa',
    '!gold|!hybrid|!bronze': 'green_only_oa'
}

# Collect data for each OA status including non-OA
data_info = []
for oa_status in oa_statuses + [None]:
    filters = common_filters.copy()
    filters['is_oa'] = 'true' if oa_status is not None else 'false'
    if oa_status:
        filters['open_access.oa_status'] = oa_status
    
    url = build_url(base_url, endpoint, filters, group_by)
    print(f'{oa_status} url: {url}')
    
    data = get_data_from_api(url)
    col_name = column_names.get(oa_status, 'non_oa')
    data_info.append((data, col_name))

# Create and merge dataframes
df = create_and_merge_dfs(data_info)

In [ ]:
df_sorted = df.sort_values('publication_year', ascending=False)
df_sorted

In [ ]:
df_sorted.to_csv('works_by_oa_status.csv', index=False)

---


### <a id="S2">S2: Which journals have we published in the most over the last 10 years?

#### HKUST journal articles, 2014-2023, by source (journal)
Reference: [openalex-api-tutorials](https://github.com/ourresearch/openalex-api-tutorials/blob/main/notebooks/institutions/uw-collaborators.ipynb)

In [ ]:
ust_id = "https://ror.org/00q4vv597"
email = "#your email"

# specify endpoint
endpoint = 'works'

# build the 'filter' parameter
filters = ",".join((
    f'institutions.ror:{ust_id}',
    'is_paratext:false',
    'type_crossref:journal-article', 
    'publication_year:2014-2023'
))

# put the URL together
filtered_works_url = f'https://api.openalex.org/{endpoint}?filter={filters}'

if email:
    filtered_works_url += f"&mailto={email}"
print(f'complete URL with filters:\n{filtered_works_url}')

In [ ]:
import requests
r = requests.get(filtered_works_url)
results_page = r.json()
print(f"works retrieved: \n{results_page['meta']['count']}")

In [ ]:
# there are 25 results per page
results_page['meta']['count'] / 25

In [ ]:
cursor = '*'

select = ",".join((
    'id',
    'ids',
    'title',
    'display_name',
    'publication_year',
    'publication_date',
    'primary_location',
    'open_access',
    'authorships',
    'cited_by_count',
    'is_retracted',
    'is_paratext',
))

# loop through pages
works = []
loop_index = 0
while cursor:
    
    # set cursor value and request page from OpenAlex
    url = f'{filtered_works_url}&select={select}&cursor={cursor}'
    page_with_results = requests.get(url).json()
    
    results = page_with_results['results']
    works.extend(results)

    # update cursor to meta.next_cursor
    cursor = page_with_results['meta']['next_cursor']
    loop_index += 1
    if loop_index in [5, 10, 20, 50, 100] or loop_index % 500 == 0:
        print(f'{loop_index} api requests made so far')
print(f'done. made {loop_index} api requests. collected {len(works)} works')

In [ ]:
# save raw data to pickle
import pickle

# uncomment these lines and run to save the results so we won't have to fetch them
# again next time we run the notebook

# pickle file in the current folder
import os
with open('hkust_j_2014-2023.pickle', 'wb') as outf:
    pickle.dump(works, outf, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# OR uncomment these lines and run to load the saved results
with open('hkust_j_2014-2023.pickle', 'rb') as f:
    works = pickle.load(f)

In [ ]:
# works[0]

In [ ]:
import pandas as pd

data = []

for work in works:
    work_id = work.get('id')
    work_ids = work.get('ids', {})
    work_doi = work_ids.get('doi')
    work_publication_year = work.get('publication_year')
    work_title = work.get('title')

    # Initialize other variables that might be populated from different fields
    source_display_name = None
    host_organization_name = None
    host_organization_lineage_names = None

    # Check if primary_location and source exist and are not None
    primary_location = work.get('primary_location')
    if primary_location:
        source = primary_location.get('source')
        if source:
            source_display_name = source.get('display_name')
            issn_l = source.get('issn_l')
            issn_value = source.get('issn', [])
            issn = "; ".join(issn_value) if isinstance(issn_value, (list, tuple)) else str(issn_value)
            host_organization_name = source.get('host_organization_name')
            host_organization_lineage_names = ", ".join(source.get('host_organization_lineage_names', []))

    # Append a dictionary with the required information to data list
    data.append({
        'openalex_id': work_id,
        # 'title': work_title,
        'pub_year': work_publication_year,
        'journal': source_display_name,
        'issn_l': issn_l,
        'publisher': host_organization_name,
        'publisher_full': host_organization_lineage_names,
        'doi': work_doi
    })

df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
# Create the pivot table for counts of openalex_id
pivot_table = df.pivot_table(index='journal', columns='pub_year', 
                             values='openalex_id', aggfunc='count', fill_value=0)

# Assuming each journal has a single publisher, we take the first publisher for each journal
publishers = df.drop_duplicates(subset='journal').set_index('journal')['publisher']

# Join the pivot table with the publisher series
final_df = pivot_table.join(publishers)

In [ ]:
def normalize_publisher_name(publisher):
    if publisher is None:
        return publisher

    mapping = {
        "SpringerNature": ["springer", "nature", "biomed central", "bmc"],
        "Elsevier": ["elsevier", "RELX"],
        "Wiley": ["wiley", "hindawi"],
        "IEEE": ["IEEE", "Institute of Electrical and Electronics Engineers"],
        "MDPI": ["Multidisciplinary Digital Publishing Institute", "mdpi"], 
        "Taylor & Francis": ["Routledge", "Francis"],
        "SAGE": ["SAGE"],
        "De Gruyter": ["De Gruyter"],
        "Emerald": ["Emerald"],
        "CUP": ["cambridge university press", "cup"],
        "OUP": ["oxford university press", "oup"],
        "ACS": ["American Chemical Society"],
        "RSC": ["Royal Society of Chemistry"],
        "ACM": ["Association for Computing Machinery"],
        "APS": ["American Physical Society"],
        "IOP": ["IOP", "Institute of Physics"],
        "ASCE": ["American Society of Civil Engineers"],
        "IET": ["Institution of Engineering and Technology"],
        "AOM": ["Academy of Management"],
        "AAAS": ["American Association for the Advancement of Science"],
        "INFORMS": ["Institute for Operations Research and the Management Sciences"],
        "Frontiers": ["Frontiers"],
        "eLife": ["eLife"],        
        "PLoS": ["Public Library of Science"],  
    }
    
    publisher_lower = publisher.lower()
    for normalized_name, variants in mapping.items():
        if any(variant.lower() in publisher_lower for variant in variants):
            return normalized_name
    return publisher  # Return the original name if no match is found

# Now apply the map function with the updated normalize_publisher_name function
publishers = publishers.map(normalize_publisher_name)

# Join the pivot table with the normalized publishers Series
final_df = pivot_table.join(publishers)

# Sort the DataFrame by the year 2023 in descending order
sorted_final_df = final_df.sort_values(by=2023, ascending=False)

In [ ]:
sorted_final_df.head()

In [ ]:
sorted_final_df.to_csv('j_by_source_publisher.csv', index=True)

----

### <a id="S3">S3: How has our contribution to the Sustainable Development Goals (SDG) evolved over the years?

#### HKUST journal + conference articles, 2014-2023, by SDG areas

In [ ]:
import requests
import pandas as pd

# Create a function to build the API URL
def build_url(ror_id, pub_year, publication_type, group_by=None):
    # specify endpoint
    endpoint = 'works'

    # build the base 'filter' parameter list with required filters
    filters = [
        f'institutions.ror:{ror_id}',
        f'publication_year:{pub_year}',
        f'type:{publication_type}',
        'is_paratext:false'
    ]
    
    # join the filter elements into a single string
    filters_str = ",".join(filters)

    # put the URL together
    filtered_works_url = f'https://api.openalex.org/{endpoint}?filter={filters_str}'

    # add 'group_by' to URL if provided
    if group_by:
        filtered_works_url += f'&group_by={group_by}'

    return filtered_works_url

# Create a function to get data from the API
def get_data_from_api(url):
    response = requests.get(url)
    data = response.json()
    if 'group_by' in data:
        return data['group_by']
    return data or []

In [ ]:
# Set parameters
ror_id = '00q4vv597'
publication_type = 'article'
group_by = 'sustainable_development_goals.id'
years = range(2014, 2024)

In [ ]:
from functools import reduce

# Create a function to create a dataframe
def create_df(data, col_name):
    df = pd.DataFrame(data)[['key', 'key_display_name', 'count']]
    df = df.rename(columns={'key_display_name': 'SDG', 'count': col_name})
    df = df.set_index('key')
    return df.fillna(0)

# Loop through each year, get data and merge into a single dataframe
dfs = []
for year in years:
    url = build_url(ror_id, year, publication_type, group_by)
    data = get_data_from_api(url)
    df = create_df(data, str(year))
    dfs.append(df)
    
df = reduce(lambda left, right: pd.merge(left, right, on='SDG', how='outer'), dfs)

In [ ]:
df_sorted_2023 = df.sort_values('2023', ascending=False)
df_sorted_2023

In [ ]:
# Keep the first 7 rows
top_rows = df_sorted_2023.iloc[:7]

# Sum the rest of the rows
other_sdg_data = df_sorted_2023.iloc[7:].sum().to_frame().T

# Concatenate the top rows with the summed 'Other SDGs' row
result_df = pd.concat([top_rows, other_sdg_data], ignore_index=True)
result_df

In [ ]:
df_trans = result_df.transpose()

new_header = df_trans.iloc[0]
df_trans = df_trans[1:]
df_trans.columns = new_header
df_new = df_trans.reset_index()
df_new.rename(columns={'index': 'publication_year', df_new.columns[-1]: 'Other SDGs'}, inplace=True)

In [ ]:
# Get all HKUST articles by year
ror_id = '00q4vv597'
publication_type = 'article'
pub_year = '2014-2023'
group_by = 'publication_year'

url = build_url(ror_id, pub_year, publication_type, group_by)
data = get_data_from_api(url)

In [ ]:
df_all = pd.DataFrame(data)[['key', 'count']].rename(columns={'key': 'publication_year', 'count': 'all articles'})
df_all

In [ ]:
# Count Non SDGs articles by year
merged_df = pd.merge(df_new, df_all, on='publication_year', how='outer') 

merged_df['Non SDGs'] = merged_df['all articles'] - merged_df.iloc[:, 1:9].sum(axis=1)
merged_df

In [ ]:
merged_df.to_csv('works_by_sdg.csv', index=False)

---

### <a id="S4">S4: Which of our publications have had the most impact over the years?

#### HKUST journal  & conference articles, 2014-2023, by citations
Reference: [openalex-api-tutorials](https://github.com/ourresearch/openalex-api-tutorials/blob/main/notebooks/institutions/uw-collaborators.ipynb)

In [ ]:
ust_id = "https://ror.org/00q4vv597"
email = "#your email"

# specify endpoint
endpoint = 'works'

# build the 'filter' parameter
filters = ",".join((
    f'institutions.ror:{ust_id}',
    'is_paratext:false',
    'type:article', 
    'publication_year:2014-2023'
))

# put the URL together
filtered_works_url = f'https://api.openalex.org/{endpoint}?filter={filters}'

if email:
    filtered_works_url += f"&mailto={email}"
print(f'complete URL with filters:\n{filtered_works_url}')

In [ ]:
import requests
r = requests.get(filtered_works_url)
results_page = r.json()
print(f"works retrieved: \n{results_page['meta']['count']}")

In [ ]:
# there are 25 results per page
results_page['meta']['count'] / 25

In [ ]:
cursor = '*'

select = ",".join((
    'id',
    'ids',
    'title',
    'display_name',
    'publication_year',
    'publication_date',
    'primary_location',
    'open_access',
    'authorships',
    'cited_by_count',
    'counts_by_year',
    'is_retracted',
    'is_paratext',
    'concepts',
))

# loop through pages
works = []
loop_index = 0
while cursor:
    
    # set cursor value and request page from OpenAlex
    url = f'{filtered_works_url}&select={select}&cursor={cursor}'
    page_with_results = requests.get(url).json()
    
    results = page_with_results['results']
    works.extend(results)

    # update cursor to meta.next_cursor
    cursor = page_with_results['meta']['next_cursor']
    loop_index += 1
    if loop_index in [5, 10, 20, 50, 100] or loop_index % 250 == 0:
        print(f'{loop_index} api requests made so far')
print(f'done. made {loop_index} api requests. collected {len(works)} works')

In [ ]:
# save raw data to pickle
import pickle

# uncomment these lines and run to save the results so we won't have to fetch them
# again next time we run the notebook

# # pickle file in the current folder
# import os
# with open('hkust_2014-2023_cites_concepts.pickle', 'wb') as outf:
#     pickle.dump(works, outf, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# OR uncomment these lines and run to load the saved results
with open('hkust_2014-2023_cites_concepts.pickle', 'rb') as f:
    works = pickle.load(f)

In [ ]:
# works[0]

In [ ]:
import pandas as pd

# Define your concept of interest
# concept_of_interest = 'artificial intelligence'  # Set to None to include all works
concept_of_interest =  None

data = []

for work in works:
    # If a concept of interest is specified, check for it; otherwise, include all works.
    if concept_of_interest is None or any(concept.get('display_name') == concept_of_interest for concept in work.get('concepts', [])):
        work_id = work.get('ids', {}).get('openalex')
        work_doi = work.get('ids', {}).get('doi')
        work_publication_year = work.get('publication_year')
        work_title = work.get('title')
        cited_by_count = work.get('cited_by_count')
        counts_by_year_dict = {count['year']: count['cited_by_count'] for count in work.get('counts_by_year', [])}

        # Initialize columns for each year
        year_columns = {str(year): counts_by_year_dict.get(year, 0) for year in range(2014, 2024)}
        
        # Get the first author's name and surname, if available
        authorships = work.get('authorships', [])
        first_author_name = authorships[0]['author']['display_name'] if authorships else None
        
        # Extract the surname by splitting the name and taking the last part
        first_author_surname = first_author_name.split()[-1] if first_author_name else None

        # Concatenate surname with publication year to form author_year
        author_year = f"{first_author_surname}_{work_publication_year}" if first_author_surname else None
        # Initialize other variables that might be populated from different fields
        source_display_name = None

        # Check if primary_location and source exist and are not None
        primary_location = work.get('primary_location')
        if primary_location:
            source = primary_location.get('source')
            if source:
                source_display_name = source.get('display_name')

        # Append a dictionary with the required information to data list
        data.append({
            'openalex_id': work_id,
            'first_author': first_author_name,
            'author_year': author_year,
            'title': work_title,
            'pub_year': work_publication_year,
            'journal': source_display_name,
            'doi': work_doi,
            'cited_by_count': cited_by_count,
            **year_columns  # Unpack the year_columns dictionary into the data dictionary
        })

# Create a DataFrame from the data list
df = pd.DataFrame(data)
df.head()

In [ ]:
# Count no. of works with total citations over 10
cites_over_10 = len(df[df['cited_by_count'] >= 10])
cites_over_10 

In [ ]:
df_cites_over_10 = df[df['cited_by_count'] >= 10]
df_cites_over_10.head()

In [ ]:
df_cites_over_10.to_csv('works_by_citations_over_10.csv', index=False, encoding='utf-8-sig')